In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# ------------------------------------------------------------------------------
#
#   Copyright 2024 Valory AG
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# ------------------------------------------------------------------------------

"""Script for listing markets."""

from collections import defaultdict
from datetime import datetime, timedelta
from enum import Enum
from string import Template
from typing import Any, Dict

import pandas as pd
import requests


class MarketState(Enum):
    """Market state"""

    OPEN = 1
    PENDING = 2
    FINALIZING = 3
    ARBITRATING = 4
    CLOSED = 5
    UNKNOWN = 6

    def __str__(self) -> str:
        """Prints the market status."""
        return self.name.capitalize()


answer_mapping = defaultdict(
    lambda: "Unknown",
    {
        "0x0000000000000000000000000000000000000000000000000000000000000000": "Yes",
        "0x0000000000000000000000000000000000000000000000000000000000000001": "No",
        "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff": "Invalid",
    },
)

headers = {
    "Accept": "application/json, multipart/mixed",
    "Content-Type": "application/json",
}

omen_xdai_fpmms_query = Template(
    """
    {
      fixedProductMarketMakers(
        where: {
          creator: "0x89c5cc945dd550bcffb72fe42bff002429f46fec",
          creationTimestamp_gt: "${creationTimestamp_gt}"
        }
        orderBy: creationTimestamp
        orderDirection: asc
        first: 1000
      ) {
        question {
          title
          outcomes
          currentAnswer
          currentAnswerTimestamp
          answers {
            answer
          }
        }
        id
        openingTimestamp
        resolutionTimestamp
        creationTimestamp
        isPendingArbitration
        answerFinalizedTimestamp
        currentAnswer
      }
    }
    """
)


def _get_date(timestamp: float) -> str:
    try:
        timestamp = int(timestamp)
        return datetime.utcfromtimestamp(timestamp).isoformat()
    except Exception:  # pylint: disable=broad-except
        return ""


def _get_market_state(market: Dict[str, Any]) -> MarketState:
    try:
        now = datetime.utcnow().timestamp()
        market_status = MarketState.CLOSED
        if market["currentAnswer"] is None and now >= float(
            market.get("openingTimestamp", 0)
        ):
            market_status = MarketState.PENDING
        elif market["currentAnswer"] is None:
            market_status = MarketState.OPEN
        elif market["isPendingArbitration"]:
            market_status = MarketState.ARBITRATING
        elif now < float(market["answerFinalizedTimestamp"]):
            market_status = MarketState.FINALIZING

        return market_status
    except Exception:  # pylint: disable=broad-except
        return MarketState.UNKNOWN


def _get_remaining_resolution_time(market: Dict[str, Any]) -> str:
    if market["answerFinalizedTimestamp"]:
        now = datetime.utcnow()
        finalized_time = datetime.utcfromtimestamp(
            int(market["answerFinalizedTimestamp"])
        )
        time_diff = finalized_time - now

        if time_diff >= timedelta(0):
            hours = int(time_diff.total_seconds() // 3600)
            minutes = int((time_diff.total_seconds() % 3600) // 60)
            return "{:02d}:{:02d}".format(hours, minutes)
    return "--"


def _get_market_title(market: Dict[str, Any]) -> str:
    question_title = ""
    if market.get("question"):
        question_title = market["question"].get("title", "")
    return question_title


def _get_current_answer(market: Dict[str, Any]) -> str:
    currentAnswer = market.get("currentAnswer", "")
    if not currentAnswer:
        currentAnswer = ""
    return answer_mapping.get(currentAnswer.lower(), "Unanswered")


def _get_num_answers(market) -> int:
    question = market.get("question", {})
    if not question:
        question = {}
    return len(question.get("answers", []))


def _execute_fpmm_query() -> Dict[str, Any]:
    url = "https://api.thegraph.com/subgraphs/name/protofire/omen-xdai"

    all_markets = []
    creationTimestamp_gt = "0"

    while True:
        query = omen_xdai_fpmms_query.substitute(
            creationTimestamp_gt=creationTimestamp_gt,
        )
        content_json = {
            "query": query,
            "variables": None,
            "extensions": {"headers": None},
        }
        res = requests.post(url, headers=headers, json=content_json, timeout=300)
        result_json = res.json()
        markets = result_json.get("data", {}).get("fixedProductMarketMakers", [])

        if not markets:
            break

        all_markets.extend(markets)
        creationTimestamp_gt = markets[len(markets) - 1]["creationTimestamp"]

    output = {"data": {"fixedProductMarketMakers": all_markets}}
    return output


def _generate_markets_df() -> pd.DataFrame:
    data = _execute_fpmm_query()
    rows = []
    for entry in data["data"]["fixedProductMarketMakers"]:
        rows.append(
            {
                "Title": _get_market_title(entry),
                "Current answer": _get_current_answer(entry),
                "Num answers": _get_num_answers(entry),
                "State": _get_market_state(entry),
                "Remaining time to challenge": _get_remaining_resolution_time(entry),
                "Creation (UTC)": _get_date(entry["creationTimestamp"]),
                "Answer finalized (UTC)": _get_date(entry["answerFinalizedTimestamp"]),
                "Resolution (UTC)": _get_date(entry["resolutionTimestamp"]),
                "URL": f'https://aiomen.eth.limo/#/{entry.get("id","")}',
            }
        )

    return pd.DataFrame(rows)

In [ ]:
from IPython.display import display, HTML
df = _generate_markets_df()

# Display finalizing markets, which can still be challenged.
df_filtered = df[df["State"] == MarketState.FINALIZING]
pd.options.display.max_colwidth = 150
display(df_filtered)